In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import csv
import re

In [2]:
URL = "https://www.espn.com/college-football/scoreboard/_/group/80/year/2019/seasontype/2/week/1"

In [3]:
def scrape_and_write_week_of_year(driver = None, writer = None, week = None, year = None):
    URL = f"https://www.espn.com/college-football/scoreboard/_/group/80/year/{year}/seasontype/2/week/{week}"
    driver.get(URL)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    soup = BeautifulSoup(driver.page_source,'html')
    
    events = soup.find('div', attrs = {'id': 'events'})
    
    for scoreboard in events.findAll('article',
                                    attrs = {'class' : re.compile("^scoreboard football")}):
        try:
            home_id = scoreboard['data-homeid']
            away_id = scoreboard['data-awayid']

            away_team = (scoreboard
                             .find('tr', attrs = {"class": "away"})
                             .find('span', attrs = {"class" : "sb-team-short"})
                             .text)

            away_score = (scoreboard
                             .find('tr', attrs = {"class": "away"})
                             .find('td', attrs = {"class" : "total"})
                             .span.text)

            home_team = (scoreboard
                             .find('tr', attrs = {"class": "home"})
                             .find('span', attrs = {"class" : "sb-team-short"})
                             .text)

            home_score = (scoreboard
                             .find('tr', attrs = {"class": "home"})
                             .find('td', attrs = {"class" : "total"})
                             .span.text)

            writer.writerow([year,week,home_team, away_team, home_id, away_id, home_score, away_score])
        except:
            pass

In [4]:
with open("../data/games.csv", "w") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Year','Week', 'HomeTeam', 'AwayTeam', 'HomeId', 'AwayId', 'HomeScore', 'AwayScore'])

In [5]:
# scrape the data from 2018
with webdriver.Firefox() as driver:
    with open("../data/games.csv", "a") as csvfile:
        writer = csv.writer(csvfile)
    
        for week in range(1,15):
            scrape_and_write_week_of_year(driver = driver, writer = writer, week = week, year = 2018)

In [6]:
# get the first week of 2019
with webdriver.Firefox() as driver:
    with open("../data/games.csv", "a") as csvfile:
        writer = csv.writer(csvfile)

        scrape_and_write_week_of_year(driver = driver, writer = writer, week = 1, year = 2019)